# Fictitious Play
This tutorial  demonstrates fictitious play (FP) on a zero-sum strategic (non-extensive) game. 

For the payoff matrix, row player is maximizer, coloumn player is minimizer.

Reference: [https://towardsdatascience.com/introduction-to-fictitious-play-12a8bc4ed1bb](https://towardsdatascience.com/introduction-to-fictitious-play-12a8bc4ed1bb)

In [3]:
import numpy as np
! pip install pandas
import pandas as pd
# GameMatrix = np.array([[2,1], [1,1]])
GameMatrix = np.array([[0,2,-1], [-1,0,1], [1,-1,0]])

Itr = 10000
print('Game matrix: ')
pd.DataFrame(GameMatrix)


Game matrix: 


0  1  2
0  0  2 -1
1 -1  0  1
2  1 -1  0

In [4]:

def fp(GameMatrix):
    """ Fictitious play on normal-from game."""
    # a random initial step on row
    row_value = np.zeros(GameMatrix.shape[0])
    col_value = np.zeros(GameMatrix.shape[1])
    min_id_list, max_id_list = [], []

    for i in range(Itr):
        # current l is row
        max_id = np.argmax(row_value) # row player is maximizer
        max_id_list.append(max_id)
        l = GameMatrix[max_id] # l is column now
        col_value += np.array(l)
        
        # current l is column
        min_id = np.argmin(col_value)  # column player is minimizer
        min_id_list.append(min_id)
        l = GameMatrix[:, min_id]  #  l is row now
        row_value += np.array(l)
        
    # The statistical frequencies of occurrence of different entries are just their probability masses in final policy,
    # which is the average best response over history.
    # hist, _ = np.histogram(max_id_list, bins=GameMatrix.shape[0])  # np.histogram is the wrong function to use!
    hist = np.bincount(max_id_list, minlength=GameMatrix.shape[0])
    max_policy = hist/np.sum(hist)
    hist = np.bincount(min_id_list, minlength=GameMatrix.shape[1])
    min_policy = hist/np.sum(hist)

    row_value = row_value / (i+1)  # historical average, row value is lower bound
    col_value = col_value / (i+1)  # historical average, column value is upper bound
    print(f'For row player, strategy is {max_policy}, game value (lower bound): {row_value}')
    print(f'For column player, strategy is {min_policy}, game value (upper bound): {col_value}')

    return (max_policy, min_policy), (row_value, col_value)

fp(GameMatrix)


For row player, strategy is [0.2496 0.3321 0.4183], game value (lower bound): [0.0811 0.0818 0.0859]
For column player, strategy is [0.3347 0.2488 0.4165], game value (upper bound): [0.0862 0.0809 0.0825]


((array([0.2496, 0.3321, 0.4183]), array([0.3347, 0.2488, 0.4165])),
 (array([0.0811, 0.0818, 0.0859]), array([0.0862, 0.0809, 0.0825])))

# Fictitious Self-Play (Normal-form Game)
Here we try to demonstrate the fictious self-play algorithm (or say a non-neural verison of Neural Fictious Self-Play, NFSP, [2]) on a zero-sum strategic (non-extensive) game.

Note that in original paper of FSP [1], the algorithm of FSP actually uses the supervised learning (SL) and reinforcement learning (RL) language to describe it, quite close to NFSP. SL is for achieving the average policy and RL is for reaching the best response policy. But here we have simple normal-form game with utility written in matrix, so it is a non-neural version. For the average policy, from above FP example we can see, it can be actually directly estimated with the historical frequency of actions for each player. Best response policy is easily calculated since it should be deterministic.

Also note that it is not the XFP algorithm for extensive-form games.

Refereneces:

[1] FSP http://proceedings.mlr.press/v37/heinrich15.html

[2] NFSP https://arxiv.org/abs/1603.01121

In [5]:
import numpy as np
! pip install pandas
import pandas as pd
# GameMatrix = np.array([[-2,3], [3,-4]])
GameMatrix = np.array([[0,2,-1], [-1,0,1], [1,-1,0]])

print('Game matrix: ')
pd.DataFrame(GameMatrix)


Game matrix: 


0  1  2
0  0  2 -1
1 -1  0  1
2  1 -1  0

In [6]:
# hyperparameters for FSP
Eta = 0.2 # eta=1, only best response policy as FP; eta=0, only average policy; see either [1] or [2]
Avg_warmup = 0.2 # start to get average policy after certain ratio of overall iterations
Itr = 10000

def sample_from_categorical(dist):
    """
    sample once from a categorical distribution, return the entry index.
    dist: should be a list or array of probabilities for a categorical distribution
    """
    return np.argmax(np.random.multinomial(1, dist))

def fsp(GameMatrix):
    """ Fictitious self-play on normal-from game."""
    # a random initial step on row
    row_value = np.zeros(GameMatrix.shape[0])
    col_value = np.zeros(GameMatrix.shape[1])
    min_id_list, max_id_list = [], []

    # initialize the average policy as uniform
    max_policy = 1./GameMatrix.shape[0]*np.ones(GameMatrix.shape[0])
    min_policy = 1./GameMatrix.shape[1]*np.ones(GameMatrix.shape[1])

    for i in range(Itr):
        # current l is row
        if i< Avg_warmup or np.isnan(max_policy).any() or sample_from_categorical([1-Eta, Eta]): # if True, the case under Eta, best response (greedy) policy
            max_id = np.argmax(row_value) # row player is maximizer
            max_id_list.append(max_id)  # the calculation of average policy only uses actions from the best response policy
        else:  # sample action from the average policy
            max_id = sample_from_categorical(max_policy)
        l = GameMatrix[max_id] # l is column now
        col_value += np.array(l)
        
        # current l is column
        if i< Avg_warmup or np.isnan(min_policy).any() or sample_from_categorical([1-Eta, Eta]):
            min_id = np.argmin(col_value) # column player is minimizer
            min_id_list.append(min_id)
        else:
            min_id = sample_from_categorical(min_policy)
        l = GameMatrix[:, min_id]  #  l is row now
        row_value += np.array(l)
        
        # get the average policy for each player at each iteration
        hist = np.bincount(max_id_list, minlength=GameMatrix.shape[0])
        max_policy = hist/np.sum(hist)
        hist = np.bincount(min_id_list, minlength=GameMatrix.shape[1])
        min_policy = hist/np.sum(hist)
        

    # The statistical frequencies of occurrence of different entries are just their probability masses in final policy,
    # which is the average best response over history.
    hist = np.bincount(max_id_list, minlength=GameMatrix.shape[0])
    max_policy = hist/np.sum(hist)
    hist = np.bincount(min_id_list, minlength=GameMatrix.shape[1])
    min_policy = hist/np.sum(hist)

    row_value = row_value / (i+1)  # historical average
    col_value = col_value / (i+1)
    print(f'For row player, strategy is {max_policy}, game value (lower bound): {row_value}')
    print(f'For column player, strategy is {min_policy}, game value (upper bound): {col_value}')

    return (max_policy, min_policy), (row_value, col_value)

fsp(GameMatrix)

For row player, strategy is [0.2938856 0.6617357 0.0443787], game value (lower bound): [-0.3086  0.5392 -0.0462]
For column player, strategy is [0.23328306 0.07139266 0.69532428], game value (upper bound): [-0.273   0.7935 -0.0577]


((array([0.2938856, 0.6617357, 0.0443787]),
  array([0.23328306, 0.07139266, 0.69532428])),
 (array([-0.3086,  0.5392, -0.0462]), array([-0.273 ,  0.7935, -0.0577])))

it seems FSP does not work?

## 1. A modified version, never get best response against the opponent's best response, but only against the average policy.
It works.

In [54]:
# hyperparameters for FSP
Eta = 0.2 # eta=1, only best response policy as FP; eta=0, only average policy; see either [1] or [2]
Avg_warmup = 0.2 # start to get average policy after certain ratio of overall iterations
Itr = 10000  # overall update iteration
N=300  # sample number to evaluate the average policy, since single sample from the average policy will be inaccurate

def sample_from_categorical(dist):
    """
    sample once from a categorical distribution, return the entry index.
    dist: should be a list or array of probabilities for a categorical distribution
    """
    return np.argmax(np.random.multinomial(1, dist))

# a random initial step on row
row_value = np.zeros(GameMatrix.shape[0])
col_value = np.zeros(GameMatrix.shape[1])
min_id_list, max_id_list = [], []

# initialize the average policy as uniform
max_policy = 1./GameMatrix.shape[0]*np.ones(GameMatrix.shape[0])
min_policy = 1./GameMatrix.shape[1]*np.ones(GameMatrix.shape[1])

for i in range(Itr):
    print(i)
    # current l is row
    if i< Avg_warmup or np.isnan(max_policy).any() or sample_from_categorical([1-Eta, Eta]): # if True, the case under Eta, best response (greedy) policy
        max_id = np.argmax(row_value)
        max_id_list.append(max_id)  # the calculation of average policy only uses actions from the best response policy
    # sample action from the average policy for N times to evaluate the utility of it
    col_value = np.zeros(GameMatrix.shape[1])
    for j in range(N):
        max_id = sample_from_categorical(max_policy)
        l = GameMatrix[max_id] # l is column now
        col_value += np.array(l)
#         col_value = (j*col_value + np.array(l))/(j+1)
    col_value = col_value/N
    
    # current l is column
    if i< Avg_warmup or np.isnan(min_policy).any() or sample_from_categorical([1-Eta, Eta]):
        min_id = np.argmin(col_value)
        min_id_list.append(min_id)
    # sample action from the average policy for N times to evaluate the utility of it
    row_value = np.zeros(GameMatrix.shape[0])
    for j in range(N):
        min_id = sample_from_categorical(min_policy)
        l = GameMatrix[:, min_id]  #  l is row now
        row_value += np.array(l)
#         row_value = (j*row_value + np.array(l))/(j+1)
    row_value = row_value/N

    # get the average policy for each player at each iteration
    hist = np.bincount(max_id_list, minlength=GameMatrix.shape[0])
    max_policy = hist/np.sum(hist)
    hist = np.bincount(min_id_list, minlength=GameMatrix.shape[1])
    min_policy = hist/np.sum(hist)   

# The statistical frequencies of occurrence of different entries are just their probability masses in final policy,
# which is the average best response over history.
hist = np.bincount(max_id_list, minlength=GameMatrix.shape[0])
max_policy = hist/np.sum(hist)
hist = np.bincount(min_id_list, minlength=GameMatrix.shape[1])
min_policy = hist/np.sum(hist)

row_value = row_value / (i+1)  # historical average
col_value = col_value / (i+1)
print(f'For row player, strategy is {max_policy}, game value (lower bound): {row_value}')
print(f'For column player, strategy is {min_policy}, game value (upper bound): {col_value}')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## 2. Another modified version, decreasing eta value to 0 to gradually get best response policy against the average policy
It doesn't work.

In [55]:
# hyperparameters for FSP
Eta_ini = 0.2 # eta=1, only best response policy as FP; eta=0, only average policy; see either [1] or [2]
Eta_final = 0
Avg_warmup = 0.2 # start to get average policy after certain ratio of overall iterations
Itr = 10000

def sample_from_categorical(dist):
    """
    sample once from a categorical distribution, return the entry index.
    dist: should be a list or array of probabilities for a categorical distribution
    """
    return np.argmax(np.random.multinomial(1, dist))

# a random initial step on row
row_value = np.zeros(GameMatrix.shape[0])
col_value = np.zeros(GameMatrix.shape[1])
min_id_list, max_id_list = [], []

# initialize the average policy as uniform
max_policy = 1./GameMatrix.shape[0]*np.ones(GameMatrix.shape[0])
min_policy = 1./GameMatrix.shape[1]*np.ones(GameMatrix.shape[1])

for i in range(Itr):
    Eta = Eta_ini + (Eta_final - Eta_ini)*i/Itr  # decreasing eta
    print(i, Eta)
    # current l is row
    if i< Avg_warmup or np.isnan(max_policy).any() or sample_from_categorical([1-Eta, Eta]): # if True, the case under Eta, best response (greedy) policy
        max_id = np.argmax(row_value)
        max_id_list.append(max_id)  # the calculation of average policy only uses actions from the best response policy
    else:  # sample action from the average policy
        max_id = sample_from_categorical(max_policy)
    l = GameMatrix[max_id] # l is column now
    col_value += np.array(l)
    
    # current l is column
    if i< Avg_warmup or np.isnan(min_policy).any() or sample_from_categorical([1-Eta, Eta]):
        min_id = np.argmin(col_value)
        min_id_list.append(min_id)
    else:
        min_id = sample_from_categorical(min_policy)
    l = GameMatrix[:, min_id]  #  l is row now
    row_value += np.array(l)
    
    # get the average policy for each player at each iteration
    hist = np.bincount(max_id_list, minlength=GameMatrix.shape[0])
    max_policy = hist/np.sum(hist)
    hist = np.bincount(min_id_list, minlength=GameMatrix.shape[1])
    min_policy = hist/np.sum(hist)   
    

# The statistical frequencies of occurrence of different entries are just their probability masses in final policy,
# which is the average best response over history.
hist = np.bincount(max_id_list, minlength=GameMatrix.shape[0])
max_policy = hist/np.sum(hist)
hist = np.bincount(min_id_list, minlength=GameMatrix.shape[1])
min_policy = hist/np.sum(hist)   

row_value = row_value / (i+1)  # historical average
col_value = col_value / (i+1)
print(f'For row player, strategy is {max_policy}, game value (lower bound): {row_value}')
print(f'For column player, strategy is {min_policy}, game value (upper bound): {col_value}')


0 0.2
1 0.19998000000000002
2 0.19996
3 0.19994
4 0.19992000000000001
5 0.19990000000000002
6 0.19988
7 0.19986
8 0.19984000000000002
9 0.19982
10 0.1998
11 0.19978
12 0.19976000000000002
13 0.19974
14 0.19972
15 0.19970000000000002
16 0.19968000000000002
17 0.19966
18 0.19964
19 0.19962000000000002
20 0.1996
21 0.19958
22 0.19956000000000002
23 0.19954000000000002
24 0.19952
25 0.1995
26 0.19948000000000002
27 0.19946
28 0.19944
29 0.19942000000000001
30 0.19940000000000002
31 0.19938
32 0.19936
33 0.19934000000000002
34 0.19932
35 0.1993
36 0.19928
37 0.19926000000000002
38 0.19924
39 0.19922
40 0.19920000000000002
41 0.19918000000000002
42 0.19916
43 0.19914
44 0.19912000000000002
45 0.1991
46 0.19908
47 0.19906000000000001
48 0.19904000000000002
49 0.19902
50 0.199
51 0.19898000000000002
52 0.19896
53 0.19894
54 0.19892
55 0.19890000000000002
56 0.19888
57 0.19886
58 0.19884000000000002
59 0.19882000000000002
60 0.1988
61 0.19878
62 0.19876000000000002
63 0.19874
64 0.19872
65 0.19